In [ ]:
from bs4 import BeautifulSoup
import requests
import json
import os 

url = ''
# Send a request to the URL
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find all doctor sections

doctor_sections = soup.find_all('div', class_='et_pb_team_member')
x
hospital_name = "South City Hospital"
# Extract information
doctors = []
for section in doctor_sections:
    name = section.find('h4', class_='et_pb_module_header').text.strip()
    speciality = section.find_all('strong')[0].find_next_sibling(text=True).strip()
    education = section.find_all('strong')[1].find_next_sibling(text=True).strip()
    work_days = section.find_all('strong')[2].find_next_sibling(text=True).strip()
    image_url = section.find('img')['src']

    doctor_info = {
        'name': name,
        'speciality': speciality,
        'education': education,
        'work_days': work_days,
        'image': image_url,
        'hospital': hospital_name
    }
    doctors.append(doctor_info)

doctor_info = 'doctors_info.json'

# save i json format 
if os.path.exists(doctor_info):
    with open(doctor_info, 'r') as json_file:
        existing_data = json.load(json_file)
else:
    existing_data = []

# Append new data to existing data
existing_data.extend(doctors)

# Write updated data to the JSON file
with open(doctor_info, 'w') as json_file:
    json.dump(existing_data, json_file, indent=4)
# Print extracted information
